## A python file to quickly generate simple FFmpeg grids. 

In the example below, the FFmpeg command it generates will be 3x3 grid with files named in the following format. Just edit the forms to quickly set your desired format.

### Output example format

| column #1  |  column #2       |  column #3|
|----------|:-------------|:------|
| 0.mp4 |  1.mp4 | 2.mp4 |
| 3.mp4 |    4.mp4   |  5.mp4 |
| 6.mp4 | 7.mp4 | 8.mp4 |


There is a filter script that will also be generated with the file specified from the inputs. This is generated because with increasingly large inputs, the command for the console will be too large. This fixes that problem. You will need to move this script to the same directory that the FFmpeg command is ran from in the terminal.

### Example of 100 videos in a 10x10 grid
[link to video](https://youtu.be/CWhya8R--UI)

[![co.png](https://i.imgur.com/c294lSN.png)](https://www.youtube.com/watch?v=CWhya8R--UI)

In [10]:
#@title Inputs for script
amount_of_items_in_grid =  9#@param {type:"integer"}
filetype_of_items = "mp4" #@param {type:"string"}
amount_of_columns_on_the_horizontal_axis =  3#@param {type:"integer"}
amount_of_rows_on_the_vertical_axis =  3#@param {type:"integer"}
size_of_each_video_on_x_axis =  360#@param {type:"integer"}
size_of_each_video_on_y_axis =  360#@param {type:"integer"}
file_to_save_the_filter_for_the_command = "filter_script.txt" #@param {type:"string"}
output_file_name = "out.mp4" #@param {type:"string"}

import math
from google.colab import files

def createCommand(inputAmount, videoFileFormat, gridX, gridY, newVidSizeX, newVidSizeY, filterFile, outFile):
    
    # inputs are formatted here for the command
    def createVideoInputs(toAmount, fileFormat):
        temp = ""
        
        for i in range(toAmount):
            temp += " -i " + str(i) + "." + fileFormat + " "
        
        return temp
     
    def createFilter(inputAmount, gridX, gridY, newVidSizeX, newVidSizeY, filterFile):
    
        filterBaseArg = " -filter_complex_script "
    
        # the resolution is determined by the size of the videos
        def defineBase(finalVideoX, finalVideoY, clipsOnX, clipsOnY):
            return " nullsrc=size=" + str(finalVideoX * clipsOnX) + "x" + str(finalVideoY * clipsOnY) + " [base]; "

        def tagInputs(toAmount, x, y):
            
            inputs = ""
            
            for i in range(0, toAmount):
                inputs += " [" + str(i) + ":v] "
                inputs += " setpts=PTS-STARTPTS, "
                inputs += " scale=" + str(x) + "x" + str(y) + " " 
                inputs += " [clip" + str(i) + "]; "
            
            return inputs


        def createVideoOverlays(toAmount, gridX, gridY, vidX, vidY):
            
            overlays = " [base][clip0] overlay=shortest=1 [temp1]; "
            
            for i in range(1, toAmount):
                overlays += "[temp" + str(i) + "][clip" + str(i) + "] overlay=shortest=1"
                overlays += (":x = " + str((i % gridX) * vidX)) if i % gridX > 0 else ""
                overlays += (":y = " + str(math.floor((i) / gridY) * vidY)) if i >= gridY else ""
                overlays += " [temp" + str(i + 1) + "]; " if i != (toAmount - 1) else ""
                
            return overlays
            
        
        tempFile = open(filterFile, "w")
        
        tempFile.write(defineBase(newVidSizeX, newVidSizeY, gridX, gridY) + tagInputs(inputAmount, newVidSizeX, newVidSizeY) + createVideoOverlays(inputAmount, gridX, gridY, newVidSizeX, newVidSizeY))

        tempFile.flush()
        tempFile.close()
        
        files.download(filterFile) 

        return filterBaseArg + "\"" + filterFile + "\" "
    
    def createOutput(outFile):
        return " -c:v libx264 " + outFile

    
    return "ffmpeg " + createVideoInputs(inputAmount, videoFileFormat) + createFilter(inputAmount, gridX, gridY, newVidSizeX, newVidSizeY, filterFile) + createOutput(outFile)


# amount of items in grid
# filetype of items
# amount of columns on the horizontal axis
# amount of rows on the vertical axis
# size of each video - x-axis
# size of each video - y-axis
# file to save the filter for the command
# output file name
print(createCommand(amount_of_items_in_grid, filetype_of_items, amount_of_columns_on_the_horizontal_axis, amount_of_rows_on_the_vertical_axis, size_of_each_video_on_x_axis, size_of_each_video_on_y_axis, file_to_save_the_filter_for_the_command, output_file_name))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ffmpeg  -i 0.mp4  -i 1.mp4  -i 2.mp4  -i 3.mp4  -i 4.mp4  -i 5.mp4  -i 6.mp4  -i 7.mp4  -i 8.mp4  -filter_complex_script "filter_script.txt"  -c:v libx264 out.mp4
